In [84]:
import pandas as pd

df = pd.read_csv("../input/yttrends/growth_df_btw_ids21.tsv", sep='\t')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [85]:
import numpy as np

numeric_data = df.select_dtypes([np.number])
numeric_features = numeric_data.columns

features = [item for item in numeric_features if not item.endswith("22")]

In [86]:
target_4 = ["views_22", "comments_22", "likes_22", "dislikes_22"]
target_views = ["views_22"]
target_likes = ["likes_22"]
target_dislikes = ["dislikes_22"]
target_comments = ["comments_22"]

In [129]:
from sklearn.model_selection import train_test_split

X = df[features]
y = df[target_views]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

Просмотры:

In [130]:
correlations_views = X_train.corrwith(y_train["views_22"]).sort_values(ascending=False)
corr_df = pd.DataFrame(data={"features": correlations_views.index, "importance": correlations_views.values})
important_features = list(corr_df[np.abs(corr_df.importance) >= 0.58]["features"].values)

Лайки:

In [97]:
correlations_likes = X_train.corrwith(y_train["likes_22"]).sort_values(ascending=False)
corr_df = pd.DataFrame(data={"features": correlations_likes.index, "importance": correlations_likes.values})
important_features = list(corr_df[np.abs(corr_df.importance) >= 0.25]["features"].values)

Дизлайки:

In [105]:
correlations_dislikes = X_train.corrwith(y_train["dislikes_22"]).sort_values(ascending=False)
corr_df = pd.DataFrame(data={"features": correlations_dislikes.index, "importance": correlations_dislikes.values})
important_features = list(corr_df[np.abs(corr_df.importance) >= 0.15]["features"].values)

Комментарии:

In [116]:
correlations_comments = X_train.corrwith(y_train["comments_22"]).sort_values(ascending=False)
corr_df = pd.DataFrame(data={"features": correlations_comments.index, "importance": correlations_comments.values})
important_features = list(corr_df[np.abs(corr_df.importance) >= 0.05]["features"].values)

In [131]:
X = df[important_features]

In [132]:
y = df[target_views]

In [ ]:
y = df[target_likes]

In [107]:
y = df[target_dislikes]

In [118]:
y = df[target_comments]

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
X.columns

In [133]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.transform(y_test)

In [11]:
!pip install catboost

In [35]:
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

model = CatBoostRegressor(iterations=8000,
                             learning_rate=0.02,
                             depth=10,
                             random_seed=42,
                             task_type='GPU',
                             bagging_temperature=0.2,
                             od_type='Iter',
                             metric_period=400,
                             od_wait=100,
                             )
model.fit(X_train_scaled, y_train_scaled, early_stopping_rounds=400)
y_pred = model.predict(X_test_scaled)

0:	learn: 0.9953104	total: 107ms	remaining: 14m 15s
400:	learn: 0.4454080	total: 16.5s	remaining: 5m 12s
800:	learn: 0.2459932	total: 39s	remaining: 5m 50s
1200:	learn: 0.1989821	total: 1m	remaining: 5m 42s
1600:	learn: 0.1659364	total: 1m 14s	remaining: 4m 58s
2000:	learn: 0.1498402	total: 1m 36s	remaining: 4m 50s
2400:	learn: 0.1242555	total: 2m 7s	remaining: 4m 56s
2800:	learn: 0.1085358	total: 2m 43s	remaining: 5m 3s
3200:	learn: 0.1013265	total: 3m 5s	remaining: 4m 38s
3600:	learn: 0.1001098	total: 3m 16s	remaining: 3m 59s
4000:	learn: 0.0985644	total: 3m 29s	remaining: 3m 29s
4400:	learn: 0.0978069	total: 3m 44s	remaining: 3m 3s
4800:	learn: 0.0970798	total: 3m 57s	remaining: 2m 38s
5200:	learn: 0.0963013	total: 4m 10s	remaining: 2m 14s
5600:	learn: 0.0952378	total: 4m 27s	remaining: 1m 54s
6000:	learn: 0.0932210	total: 4m 56s	remaining: 1m 38s
6400:	learn: 0.0925789	total: 5m 10s	remaining: 1m 17s
6800:	learn: 0.0924499	total: 5m 21s	remaining: 56.6s
7200:	learn: 0.0923470	total

In [37]:
y_pred = model.predict(X_test_scaled)
print("Test R2 = %.8f" % r2_score(y_test_scaled, y_pred))

Test R2 = 0.70385731


In [14]:
model.save_model("views_model",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.81325878

In [23]:
model.save_model("likes_model",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.77273126

In [30]:
model.save_model("dislikes_model",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.81136465

In [38]:
model.save_model("comments_model",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.70385731

In [ ]:
!pip install shap

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from catboost import CatBoostRegressor, cv, Pool

from collections import defaultdict

from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

import shap
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import Pipeline

from tqdm import tqdm

In [80]:
cb = CatBoostRegressor(task_type='GPU', random_seed=42, verbose=200)

In [81]:
cb.fit(X_train_scaled, y_train_scaled)
y_pred = cb.predict(X_test_scaled)

Learning rate set to 0.072007
0:	learn: 0.9828173	total: 18.7ms	remaining: 18.7s
200:	learn: 0.3184031	total: 4.13s	remaining: 16.4s
400:	learn: 0.2326946	total: 7.21s	remaining: 10.8s
600:	learn: 0.1746617	total: 10.6s	remaining: 7.03s
800:	learn: 0.1492419	total: 13.8s	remaining: 3.43s
999:	learn: 0.1409728	total: 17s	remaining: 0us


In [82]:
print("Test R2 = %.8f" % r2_score(y_test_scaled, y_pred))

Test R2 = 0.73462535


In [58]:
cb.save_model("views_cb",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.88377347

In [66]:
cb.save_model("likes_cb",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.79123153

In [75]:
cb.save_model("dislikes_cb",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.73913935

In [83]:
cb.save_model("comments_cb",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.73462535

In [ ]:
# params = {'loss_function':'RMSE', 'verbose': 200, 'random_seed': 13, 'task_type':'GPU'}

In [ ]:
# catboost_hyperparams = { 'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
#                          'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
#                          'colsample_bylevel': hp.choice('colsample_bylevel', np.arange(0.3, 0.8, 0.1)),
#                          'n_estimators': 100, 'eval_metric': 'RMSE'}

# catbooast_fit_params = { 'early_stopping_rounds': 10, 'verbose': False }

# catboost_params = dict()
# catboost_params['reg_params'] = catboost_hyperparams
# catboost_params['fit_params'] = catbooast_fit_params
# catboost_params['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

In [ ]:
# class HPOptimiser(object):
#     """ Class to optimiser hyper-parameters using hyperopt on a given
#         set of training and validation inputs and labels """

#     def __init__(self, X_train, X_val, y_train, y_val):
#         self.X_train = X_train
#         self.X_val  = X_val
#         self.y_train = y_train
#         self.y_val  = y_val

#     def process(self, fn_name, space, trials, algo, max_evals):
#         fn = getattr(self, fn_name)
#         try:
#             result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
#         except Exception as e:
#             return {'status': STATUS_FAIL,
#                     'exception': str(e)}
#         return result, trials

#     def catboost_reg(self, para):
#         reg = CatBoostRegressor(**para['reg_params'])
#         return self.train_reg(reg, para)

#     def train_reg(self, reg, params):
#         reg.fit(self.X_train, self.y_train,
#                 eval_set=[(self.X_train, self.y_train), (self.X_val, self.y_val)],
#                 **params['fit_params'])
#         pred = reg.predict(self.X_val)
#         loss = params['loss_func'](self.y_val, pred)
#         return {'loss': loss, 'status': STATUS_OK}


In [ ]:
# obj = HPOptimiser(X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled)

# catboost_opt = obj.process(fn_name='catboost_reg', space=catboost_params, 
#                            trials=Trials(), algo=tpe.suggest, max_evals=100)
# catboost_opt

In [ ]:
# catboost_opt

In [127]:
cb_learn_rate = 0.05
n_iterations = 20000
early_stop_rounds = 400

opt_catboost_params = {'iterations' : n_iterations,
                       'colsample_bylevel': 1, 
                       'learning_rate': cb_learn_rate, 
                       'max_depth': 1,
                       'bootstrap_type' : 'Bernoulli',
                       'random_strength': 1,
                       'min_data_in_leaf': 10,
                       'l2_leaf_reg': 3,
                       'loss_function' : 'RMSE', 
                       'eval_metric' : 'RMSE',
                       'grow_policy' : 'Depthwise',
                       'max_bin' : 1024, 
                       'model_size_reg' : 0,
                       'task_type' : 'GPU',
                       'od_type' : 'IncToDec',
                       'od_wait' : 100,
                       'metric_period' : 500,
                       'verbose' : 500,
                       'subsample' : 0.8,
                       'od_pval' : 1e-10,
                       'max_ctr_complexity' : 8,
                       'has_time': False,
                       'simple_ctr' : 'FeatureFreq',
                       'combinations_ctr': 'FeatureFreq',
                       'random_seed' : 13}

In [128]:
cb_reg1 = CatBoostRegressor(**opt_catboost_params)

In [134]:
cb_reg1.fit(X_train_scaled, y_train_scaled, eval_set=(X_test_scaled, y_test_scaled), 
           use_best_model=True, 
           early_stopping_rounds=early_stop_rounds)

0:	learn: 0.9727871	test: 0.7745837	best: 0.7745837 (0)	total: 10.2ms	remaining: 3m 23s
500:	learn: 0.0721807	test: 0.1736629	best: 0.1584208 (200)	total: 1.91s	remaining: 1m 14s
bestTest = 0.1584208491
bestIteration = 200
Shrink model to first 201 iterations.


In [135]:
y_pred = cb_reg1.predict(X_test_scaled)
print("Test R2 = %.8f" % r2_score(y_test_scaled, y_pred))

Test R2 = 0.96070387


In [121]:
cb_learn_rate = 0.001
n_iterations = 80000
early_stop_rounds = 2000

opt_catboost_params = {'iterations' : n_iterations,
                       'learning_rate' : cb_learn_rate,
                       'depth': 8,
                       'bootstrap_type' : 'Bernoulli',
                       'random_strength': 1,
                       'min_data_in_leaf': 10,
                       'l2_leaf_reg': 3,
                       'loss_function' : 'RMSE', 
                       'eval_metric' : 'RMSE',
                       'grow_policy' : 'Depthwise',
                       'max_bin' : 1024, 
                       'model_size_reg' : 0,
                       'task_type' : 'GPU',
                       'od_type' : 'IncToDec',
                       'od_wait' : 100,
                       'metric_period' : 500,
                       'verbose' : 500,
                       'subsample' : 0.8,
                       'od_pval' : 1e-10,
                       'max_ctr_complexity' : 8,
                       'has_time': False,
                       'simple_ctr' : 'FeatureFreq',
                       'combinations_ctr': 'FeatureFreq',
                       'random_seed' : 13}

In [122]:
cb_reg = CatBoostRegressor(**opt_catboost_params)

In [123]:
cb_reg.fit(X_train_scaled, y_train_scaled, eval_set=(X_test_scaled, y_test_scaled), 
           use_best_model=True, 
           early_stopping_rounds=early_stop_rounds)

0:	learn: 0.9996472	test: 0.5540430	best: 0.5540430 (0)	total: 19.9ms	remaining: 26m 35s
500:	learn: 0.8541451	test: 0.3942755	best: 0.3942755 (500)	total: 7.8s	remaining: 20m 37s
1000:	learn: 0.7434809	test: 0.2885733	best: 0.2885733 (1000)	total: 16.7s	remaining: 22m
1500:	learn: 0.6534060	test: 0.2212704	best: 0.2212704 (1500)	total: 24s	remaining: 20m 55s
2000:	learn: 0.5789754	test: 0.1814307	best: 0.1814307 (2000)	total: 31.7s	remaining: 20m 34s
2500:	learn: 0.5156197	test: 0.1680464	best: 0.1680464 (2500)	total: 39.1s	remaining: 20m 10s
3000:	learn: 0.4616110	test: 0.1704293	best: 0.1673199 (2594)	total: 46.5s	remaining: 19m 52s
3500:	learn: 0.4115783	test: 0.1802292	best: 0.1673199 (2594)	total: 52.9s	remaining: 19m 16s
4000:	learn: 0.3685480	test: 0.1904190	best: 0.1673199 (2594)	total: 58.7s	remaining: 18m 34s
4500:	learn: 0.3314864	test: 0.2024865	best: 0.1673199 (2594)	total: 1m 4s	remaining: 18m 3s
bestTest = 0.1673199454
bestIteration = 2594
Shrink model to first 2595 ite

In [125]:
y_pred = cb_reg.predict(X_test_scaled)
print("Test R2 = %.8f" % r2_score(y_test_scaled, y_pred))

Test R2 = 0.90889016


In [95]:
cb_reg.save_model("views_cbreg",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.96081155 # best

In [103]:
cb_reg.save_model("likes_cbreg",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.95395155

In [114]:
cb_reg.save_model("dislikes_cbreg",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.81882261

In [126]:
cb_reg.save_model("comments_cbreg",
           format="cbm",
           export_parameters=None,
           pool=None)

# Test R2 = 0.90889016